In [1]:
from pseudo_IFD_tsne import *
#from utils import load_data, MidpointNormalize, equipotential_standard_normal, plot_heatmaps
from utils import MidpointNormalize, load_data
import matplotlib.pyplot as plt
#import tikzplotlib

from tueplots import cycler, fonts, fontsizes, bundles
from tueplots.constants import markers
from tueplots.constants.color import palettes

plt.rcParams.update(cycler.cycler(color=palettes.tue_plot))
plt.rcParams.update(fonts.aistats2022_tex(family="serif"))
plt.rcParams.update(fontsizes.aistats2022())
plt.rcParams['axes.grid'] = False

In [2]:
def KL_divergence(X_flat, Y_flat, X_unflattener, Y_unflattener):
    """
    (R^nxp x R^nxp)--> R
    """
    X = X_unflattener(X_flat)
    Y = Y_unflattener(Y_flat)
    learning_rate, perplexity = (200, 30.0)
    P = x2p(X, tol=1e-5, perplexity=perplexity)
    P = (P + np.transpose(P))
    P = P / np.sum(P)      # Why don't we devide by 2N as described everywhere?
    P = np.maximum(P, 1e-12)
    Q, _ = y2q(Y)
    return np.sum(P * (np.log(P+1e-10) - np.log(Q+1e-10)))

In [3]:
#X, y = make_blobs(n_samples=50, n_features=50, centers=4, random_state=0, shuffle=False, cluster_std=[0.1, 3, 3, 3])
X, y = load_data(100)
key = random.PRNGKey(41)
#X = onp.array(random.normal(key, shape=(50, 50)))
y_guess = random.normal(key, shape=(X.shape[0], 2))
#Y_star = TSNE(n_components=2, learning_rate=200, init=onp.array(y_guess), perplexity=30).fit_transform(X)
Y_star = tsne_fwd(X, y_guess)

X_flat, X_unflattener = flatten_util.ravel_pytree(X)   # row-wise
Y_flat, Y_unflattener = flatten_util.ravel_pytree(Y_star)

primals = (X_flat, Y_flat)
KL_divergence_fun = partial(KL_divergence, X_unflattener=X_unflattener, Y_unflattener= Y_unflattener)


===> Finding 90 nearest neighbors using Annoy approximate search using euclidean distance...
   --> Time elapsed: 0.02 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.00 seconds


In [4]:
fun = lambda y: KL_divergence(X_flat, y, X_unflattener, Y_unflattener)
H = jax.hessian(fun)(Y_flat)

Computing pairwise distances...
Starting binary search
Entered binary search function


In [5]:
H_pinv = np.linalg.pinv(H, hermitian=True)

In [9]:
mixed_Jacobian = Mixed_Jacobian_x_y(KL_divergence, X, Y_star)

Computing pairwise distances...
Starting binary search
Entered binary search function


In [10]:
mixed_Jacobian

Array([[-4.19206859e-04, -1.58331764e-04, -2.78748252e-04, ...,
        -5.15662464e-08, -2.41491307e-08, -4.62240365e-07],
       [-1.32810937e-05,  2.25255877e-04, -1.68462182e-04, ...,
        -1.56484361e-07, -1.47956996e-07, -5.86169563e-07],
       [ 1.03584916e-05, -2.89319075e-07,  1.36870403e-06, ...,
         2.49618921e-08, -1.01669855e-08, -1.39819605e-08],
       ...,
       [-2.26961288e-06, -8.29196324e-06, -2.92827053e-07, ...,
        -5.38742370e-07, -1.78460894e-07,  6.18429681e-07],
       [ 4.69996905e-07,  1.17565310e-06, -8.02914315e-07, ...,
        -6.59982979e-05, -7.43321143e-05, -2.54832667e-05],
       [ 7.67369102e-09,  2.78035600e-07, -7.20468449e-08, ...,
        -6.97024007e-06, -7.75038716e-05, -2.40416521e-05]],      dtype=float32)

In [25]:
first = lambda X: jacfwd(KL_divergence_fun, argnums=1)(X, Y_flat)
_, f_vjp = vjp(first, X_flat)
_, f_jvp = jax.linearize(first, X_flat)

def mixed_Jacobian_vector_product_1(f, first, primals, v):
    X, Y = primals
    second, second_t = jvp(first, (X,), (v,)) # --> first column of Jacobian!!!
    return second, second_t

Computing pairwise distances...
Starting binary search
Entered binary search function
Computing pairwise distances...
Starting binary search
Entered binary search function


In [27]:
vmap_fun = lambda i: mixed_Jacobian_vector_product_1(KL_divergence_fun, f_jvp, (X_flat, Y_flat), i)[1]
full_mixed_j = vmap(vmap_fun)(np.eye(len(X_flat)))

In [22]:
primals = (X_flat, Y_flat)
m_j = jacfwd(jacfwd(KL_divergence_fun, 0), 1)(*primals)

Computing pairwise distances...
Starting binary search
Entered binary search function


In [22]:
test = mixed_Jacobian_vector_product_2(KL_divergence_fun, (X_flat, Y_flat), np.eye(len(Y_flat))[0])

Compute v3
Computing pairwise distances...
Starting binary search
Entered binary search function


In [19]:
second, second_t = mixed_Jacobian_vector_product_1(KL_divergence_fun, (X_flat, Y_flat), np.eye(len(X_flat))[0])

Computing pairwise distances...
Starting binary search
Entered binary search function


In [6]:
vmap_fun = lambda i: mixed_Jacobian_vector_product_1(KL_divergence_fun, (X_flat, Y_flat), i)[1]
full_mixed_j = vmap(vmap_fun)(np.eye(len(X_flat)))


Computing pairwise distances...
Starting binary search
Entered binary search function


In [6]:
M = np.eye(len(Y_flat))
time_start = time.time()
dy_dx = M_d_y_star_d_x_P(KL_divergence_fun, primals, M, H_pinv)
time_end = time.time()
print(time_end - time_start)

Compute v3
Computing pairwise distances...
Starting binary search
Entered binary search function
77.95730757713318


In [14]:
v = np.eye(len(Y_flat))[0]
time_start = time.time()
dy_dx = V_d_y_star_d_x_P(KL_divergence_fun, primals, v, H_pinv)
time_end = time.time()
print(time_end - time_start)

Compute v3
Computing pairwise distances...
Starting binary search
Entered binary search function
20.373340845108032
